In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.2 MB/s 
     |████████████████████████████████| 7.6 MB 84.5 MB/s 
     |████████████████████████████████| 182 kB 75.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.4 MB/s 
     |████████████████████████████████| 115 kB 76.3 MB/s 
     |████████████████████████████████| 212 kB 89.7 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
foldername= "/content/drive/My Drive/nlpproject/"

In [ ]:
import numpy as np 
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import datetime
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from torch.nn.modules.loss import CrossEntropyLoss

In [ ]:
class CFG:
    str_now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    basic_lr=1e-3
    train = True
    debug = False
    offline = False
    models_path = "bert-base-uncased"
    epochs = 5000
    save_all_models = False
    apex = True
    print_freq = 20
    num_workers = 4
    model = "bert-base-uncased"
    loss_func = 'SmoothL1'
    scheduler = 'cosine'
    batch_scheduler = True
    num_cycles = 0.5
    num_warmup_steps = 0
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr = 1e-6
    llrd = True
    layerwise_lr = 5e-5
    layerwise_lr_decay = 0.9
    layerwise_weight_decay = 0.01
    layerwise_adam_epsilon = 1e-6
    layerwise_use_bertadam = False
    #pooling
    pooling = 'mean' # mean, max, min, attention, weightedlayer
    layer_start = 4
    #init_weight
    init_weight = 'normal' # normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal
    #re-init
    reinit = True
    reinit_n = 1
    #adversarial
    fgm = False
    awp = False
    adv_lr = 1
    adv_eps = 0.2
    unscale = False
    eps = 1e-6
    betas = (0.9, 0.999)
    max_len = 512
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    target_cols = ['EI', 'SN', 'TF', 'JP']
    seed = 42
    cv_seed = 42
    n_fold = 4
    trn_fold = list(range(n_fold))
    batch_size = 64
    n_targets = 4
    gpu_id = 0
    device = f'cuda:{gpu_id}'
    hidden_size = 768
    init_weight="normal"
cfg=CFG()

In [ ]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained(cfg.model)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#path=foldername+"dataset2(sep).csv"
#print(dataset[0])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
def getdl(ds):
    total_len=len(ds)
    train_len=int(len(ds)*0.9)
    val_len=int((total_len-train_len)/2)
    test_len=total_len-train_len-val_len
    [train_ds, val_ds, test_ds]=torch.utils.data.random_split(ds, [train_len, val_len, test_len])
    #return (training dataloader, validation dataloader, test dataloader)
    return len(train_ds), len(val_ds), len(test_ds), DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)
    #return DataLoader(ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)

In [ ]:
input_ids=torch.load(foldername+'EI_Extraced_feature_tensors_pooleroutput.pt')
labels=torch.load(foldername+'EI_Extraced_label_tensors_pooleroutput.pt')

In [ ]:
from torch.utils.data import TensorDataset
class Extracted_ds(Dataset):
    def __init__(self, input_ids, labels):
        self.features=input_ids
        self.labels=labels
    def __len__(self):
        return (len(self.labels))
    def __getitem__(self, index):
        input_id=self.features[index]
        label=self.labels[index]
        return {"input_ids": input_id, "labels":label}

In [ ]:
ds=Extracted_ds(input_ids, labels)

In [ ]:
len_train, len_val, len_test, train_dl_EI,val_dl_EI, test_dl_EI=getdl(ds)

In [ ]:
class myModel(nn.Module):
    def __init__(self, CFG, dropoutrate):
        super().__init__()
        self.config = AutoConfig.from_pretrained(CFG.model, ouput_hidden_states = True)
        self.CFG = CFG
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(self.config.hidden_size, 256)
        self.fc2 = nn.Linear(256, 16)
        self.fc3 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(p=dropoutrate, inplace=False)
        self._init_weights(self.fc1)
        self._init_weights(self.fc2)
        self.sig = nn.Sigmoid()
        
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            if CFG.init_weight == 'normal':
                module.weight.data.normal_(mean = 0.0, std = self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            if CFG.init_weight == 'normal':
                module.weight.data.normal_(mean = 0.0, std = self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
    
    def forward(self, input_ids):
        output = self.dropout(input_ids)

        output = self.fc1(output)
        output = self.relu(output)
        output = self.dropout(output)

        output = self.fc2(output)
        output = self.relu(output)
        output = self.dropout(output)

        output = self.fc3(output)
        output = self.sig(output)

        output = torch.squeeze(output)
        return output #2 number between 0 and 1

In [ ]:
def evaluate(labels, outputs):
  answers=(outputs>=0.5)
  allcorrect=torch.sum(answers==labels)
  return allcorrect

In [ ]:
from torch.optim import lr_scheduler
from torch import nn
from torch import optim
from torch.optim import Adam
from tqdm.notebook import tqdm
def train(train_ds, eval_ds, model, epochs, cfg, type, lr):
    if torch.cuda.is_available():  
        dev = "cuda:0" 
    else:  
        dev = "cpu" 
    device = torch.device(dev)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    criterion.to(device)
    optimizer = Adam(model.parameters(), lr=lr)
    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=500, eta_min=1e-15)
    #scheduler= optim.lr_scheduler.ExponentialLR(optimizer, 0.99, last_epoch=- 1, verbose=False)

    
    for e in range(epochs):
        totaltrainloss=0
        for i,batch in enumerate(train_ds):
            optimizer.zero_grad()
            batch.to(device)
            labels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            bloss=criterion(outputs, labels)
            bloss.backward()
            optimizer.step()
            totaltrainloss+=bloss.item()
        scheduler.step()
        totalevalloss=0
        totalcorrect=0
        totaldata=0
        with torch.no_grad():
            for batch in eval_ds:
                batch.to(device)
                blabels=batch["labels"]
                outputs=model(input_ids=batch["input_ids"])
                eloss=criterion(outputs, blabels).item()
                totalevalloss+=eloss
                totalcorrect+=evaluate(blabels, outputs)
                totaldata+=len(blabels)
        totalcorrect=(totalcorrect/(totaldata))
        print("probability that our prediction of ", type, " is correct: ", totalcorrect)
        print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len_train: .3f} | Val Loss: {totalevalloss / len_val: .3f} | current lr: {scheduler.get_last_lr()}' ) 

In [ ]:
model=myModel(cfg, 0.05)

In [ ]:
train(train_dl_EI, val_dl_EI, model, epochs=cfg.epochs, cfg=cfg, type="EI", lr=1e-4)

probability that our prediction of  EI  is correct:  tensor(0.7005, device='cuda:0')
Epoch: 1 | Train Loss:  0.957 | Val Loss:  1.056 | current lr: [9.999901304280687e-05]
probability that our prediction of  EI  is correct:  tensor(0.2811, device='cuda:0')
Epoch: 2 | Train Loss:  0.957 | Val Loss:  1.055 | current lr: [9.999605221019085e-05]
probability that our prediction of  EI  is correct:  tensor(0.3571, device='cuda:0')
Epoch: 3 | Train Loss:  0.956 | Val Loss:  1.054 | current lr: [9.999111761904054e-05]
probability that our prediction of  EI  is correct:  tensor(0.4078, device='cuda:0')
Epoch: 4 | Train Loss:  0.954 | Val Loss:  1.051 | current lr: [9.998420946416515e-05]
probability that our prediction of  EI  is correct:  tensor(0.5899, device='cuda:0')
Epoch: 5 | Train Loss:  0.949 | Val Loss:  1.050 | current lr: [9.997532801828683e-05]
probability that our prediction of  EI  is correct:  tensor(0.5760, device='cuda:0')
Epoch: 6 | Train Loss:  0.946 | Val Loss:  1.047 | curr

KeyboardInterrupt: ignored